In [ ]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np

def initialize(context):
    context.security = sid(22613) # set the security to Mitsubishi
    context.window_length = 10 # number of bars or edges in a single decision tree
   
    context.classifier = RandomForestClassifier() 
    
    context.recent_prices = deque(maxlen=context.window_length + 2)
    #context.recent_value = deque(maxlen=context.window_length + 2)
    context.X = deque(maxlen=500) # Independent, or input variables
    context.Y = deque(maxlen=500) # Dependent, or output variable
    
    context.prediction = 0
    
def handle_data(context, data):
    context.recent_prices.append(data[context.security].price)
    #context.recent_value.append([data.current(context.security, 'price')])
    if len(context.recent_prices) == context.window_length + 2: # If there's enough recent price data
        changes = np.diff(context.recent_prices) > 0
        values = np.array(context.recent_prices).flatten()
        context.X.append(values[:-1])
        context.Y.append(changes[-1]) # Add dependent variable, the final change
        
        if len(context.Y) >= 200: # There needs to be enough data points to make a good model
            
            context.classifier.fit(context.X, context.Y) # Generate the model
            
            context.prediction = context.classifier.predict(values[1:]) # Predict
            
            # If prediction = 1, buy all shares affordable, if 0 sell all shares
            order_target_percent(context.security, context.prediction)
                
            record(prediction=int(context.prediction))